# Solve 2D PDE on regular & irregular domain
First we introduce all the environment variables:

In [1]:
import sys, numpy as np
sys.path.append("../src/")
import fdm.solver.fdm_solver as FDM
import util.diff_operator_expression.diff_op_expression as Expr
from util.diff_operators.impl import d2dx, d2dy, ddt, laplacian2d
from util.domain_conditions.impl.dirichlet import dirichlet_bc as Dirichlet_BC, dirichlet_rectangle as Dirichlet_rectangle
from util.domain_conditions.core.domain import domain as Domain

## Regular Domain
Let's introduce the 2D PDE that we're gonna solve:  
$\nabla^2 u = \frac{2}{(1+x)^3} + \frac{2}{(1+y)^3}$ whose boundary conditions are  
$u(x, 0) = 1+\frac{1}{1+x}$ for $x\in [0,1]$,  
$u(x,1)=\frac{1}{2}+\frac{1}{1+x}$ for $x\in[0, 1]$,  
$u(0, y) = 1+\frac{1}{1+y}$ for $y\in[0,1]$,  
$u(1, y)=\frac{1}{2}+\frac{1}{1+y}$ for $y\in[0,1]$.  
Then we define our Dirichlet boundary condition.

In [2]:
domain = Domain.domain(np.array([0, 0]), np.array([1, 1]))
inDomain = lambda x, y: 0 < x < 1 and 0 < y < 1
onBoundary = lambda x, y: abs(x-1) < np.spacing(1) or abs(x) < np.spacing(1) or abs(y-1) < np.spacing(1) or abs(y) < np.spacing(1)
getBoundaryValue = lambda x, y: 1/(1+x) + 1/(1+y) # You can verify that this gives us the boundary conditions

dirichlet = Dirichlet_BC.dirichlet_bc(inDomain, onBoundary, getBoundaryValue, domain) # Boundary condition class

Now we are able to define our solver. Note that we will define the solver on four domains which are discretized differently. And fortunately, we have an analytical solution to this problem, which is $u(x, y) = \frac{1}{1+x}+\frac{1}{1+y}$.

In [3]:
f = lambda x, y: 2/(1+x)**3 + 2/(1+y)**3 # this is the function on the right hand side of our PDE
error = [] # Use this array to record the errors of our approximation.
for n in [9, 19, 39, 79]:
    dx, dy = 1/(n+1), 1/(n+1) # each grid's length
    expression = Expr.diff_operator_expression([laplacian2d.laplacian2d(dx, dy, coefficient=1)]) # we only have a laplacian operator in our PDE
    solver = FDM.fdm_solver(expression, f, dirichlet) # Insert the operator expression, function on the right hand side and the boundary condition into our solver
    u = solver.solve(n, n) # n represents grid point numbers. In this case, we have n points in both x and y axis.
    
    # We get the approximated solution. Then we compare it with the analytical solution.
    correct_function = lambda x, y: 1/(1+x) + 1/(1+y)
    correct_solution = np.zeros(n**2)
    for j in range(0, n):
        y = (j+1)/(n+1)
        for i in range(0, n):
            x = (i+1)/(n+1)
            correct_solution[j*n+i] = correct_function(x, y)
    error.append(max(abs(correct_solution-u)))
print(" n", "max_error")
for index, n in enumerate([9, 19, 39, 79]):
    print("{:2d} {:f}".format(n, error[index]))

 n max_error
 9 0.000554
19 0.000140
39 0.000035
79 0.000009
